<a href="https://colab.research.google.com/github/HwiHwi523/Course_AI_CoLab/blob/master/%EC%84%A0%ED%98%95%EB%B6%84%EB%A5%98_16010170_%EC%9C%A4%EC%9E%AC%ED%9C%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import torch
import torch.optim as optim
import numpy as np

torch.manual_seed(1)

! curl 'https://raw.githubusercontent.com/sejongresearch/2020.Spring.AI/master/HW/zoo_class.csv' -o 'zoo_class.csv'

xy = np.loadtxt('zoo_class.csv', delimiter=',', dtype=np.float32)

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]].squeeze()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3535  100  3535    0     0  19859      0 --:--:-- --:--:-- --:--:-- 19748


In [54]:
import torch.nn.functional as F

x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

# 7가지 클래스, 16가지 특징
nb_class = 7
nb_feature = 16
nb_data = len(y_train)

# One Hot Encoding에 쓰일 W, b 초기화
W_one_hot = torch.zeros((nb_feature, nb_class), requires_grad=True)
b_one_hot = torch.zeros(1, requires_grad=True)
optimizer_one_hot = optim.SGD([W_one_hot, b_one_hot], lr = 10)

# Cross Entropy에 쓰일 W, b 초기화
W_cross_entropy = torch.zeros((nb_feature, nb_class), requires_grad = True)
b_cross_entropy = torch.zeros(1, requires_grad = True)
optimizer_cross_entropy = optim.SGD([W_cross_entropy, b_cross_entropy], lr = 10)

nb_epochs = 100000
for epoch in range(nb_epochs + 1):

  # One-hot Encoding 방식
  hypothesis = F.softmax(x_train.matmul(W_one_hot) + b_one_hot, dim = 1)

  y_one_hot = torch.zeros(nb_data, nb_class)
  y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
  cost_one_hot = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

  optimizer_one_hot.zero_grad()
  cost_one_hot.backward()
  optimizer_one_hot.step()

  # Cross Entropy 방식
  cost_cross_entropy = F.cross_entropy((x_train.matmul(W_cross_entropy) + b_cross_entropy), y_train)

  optimizer_cross_entropy.zero_grad()
  cost_cross_entropy.backward()
  optimizer_cross_entropy.step()

  if epoch % 10000 == 0:
    print('Epoch {:6d}/{}'.format(epoch, nb_epochs))
    print('- Cost (One Hot Encoding) = {}'.format(cost_one_hot))
    print('- Cost (Cross Entropy) = {}'.format(cost_cross_entropy))

Epoch      0/100000
- Cost (One Hot Encoding) = 1.9459099769592285
- Cost (Cross Entropy) = 1.9459086656570435
Epoch  10000/100000
- Cost (One Hot Encoding) = 1.1749634742736816
- Cost (Cross Entropy) = 0.00011011224705725908
Epoch  20000/100000
- Cost (One Hot Encoding) = 1.1748980283737183
- Cost (Cross Entropy) = 6.0981237766100094e-05
Epoch  30000/100000
- Cost (One Hot Encoding) = 1.17487633228302
- Cost (Cross Entropy) = 4.2495623347349465e-05
Epoch  40000/100000
- Cost (One Hot Encoding) = 1.174865484237671
- Cost (Cross Entropy) = 3.268423461122438e-05
Epoch  50000/100000
- Cost (One Hot Encoding) = 1.1748590469360352
- Cost (Cross Entropy) = 2.6595262170303613e-05
Epoch  60000/100000
- Cost (One Hot Encoding) = 1.1748548746109009
- Cost (Cross Entropy) = 2.2436663130065426e-05
Epoch  70000/100000
- Cost (One Hot Encoding) = 1.1748515367507935
- Cost (Cross Entropy) = 1.940613947226666e-05
Epoch  80000/100000
- Cost (One Hot Encoding) = 1.1748493909835815
- Cost (Cross Entropy)

In [55]:
# One Hot Encoding Prediction
hypothesis = F.softmax(x_train.matmul(W_one_hot) + b_one_hot, dim = 1)
predict_one_hot = torch.argmax(hypothesis, dim = 1)

# Cross Entropy Prediction
hypothesis = F.softmax(x_train.matmul(W_cross_entropy) + b_cross_entropy, dim = 1)
predict_cross_entropy = torch.argmax(hypothesis, dim = 1)


# Print Original Classification & Prediction
print('# y_train #')
print(y_train)
print()

print('## One Hot Encoding Prediction ##')
print(predict_one_hot)
print()

print('## Cross Entropy Prediction ##')
print(predict_cross_entropy)
print()

# Print Original Result & Prediction Results
result_one_hot = predict_one_hot.float() == y_train
result_cross_entropy = predict_cross_entropy.float() == y_train
print()

print('### One Hot Encoding Result ###')
print(result_one_hot)
print()

print('### Cross Entropy Result ###')
print(result_cross_entropy)
print()

# Print Accuracy
accuracy_one_hot = list(result_one_hot).count(True) / len(result_one_hot)
accuracy_cross_entropy = list(result_cross_entropy).count(True) / len(result_cross_entropy)
print()

print('#### One Hot Encoding Accuracy ####')
print(accuracy_one_hot)
print()

print('#### Cross Entropy Accuracy ####')
print(accuracy_cross_entropy)

# y_train #
tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        6, 3, 0, 0, 2, 6, 1, 1, 2, 6, 3, 1, 0, 6, 3, 1, 5, 4, 2, 2, 3, 0, 0, 1,
        0, 5, 0, 6, 1])

## One Hot Encoding Prediction ##
tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        5, 3, 0, 0, 2, 6, 1, 1, 2, 6, 3, 1, 0, 6, 3, 1, 5, 4, 2, 2, 3, 0, 0, 1,
        0, 5, 0, 6, 1])

## Cross Entropy Prediction ##
tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0,